### **Atención automática (Self-attention)**

La **atención automática (auto-atención)** es un mecanismo utilizado en redes neuronales que permite al modelo enfocarse en diferentes partes de los datos de entrada al generar cada parte de la salida. Es un componente clave de la arquitectura *Transformer*, ampliamente usada en tareas de procesamiento de lenguaje natural como la traducción automática, la resumición de textos y el análisis de sentimientos.

La idea detrás de la atención automática es permitir que el modelo *pese* la importancia de cada token de entrada al generar cada token de salida. Esto se logra mediante el cálculo de una suma ponderada de los tokens de entrada, donde los pesos son determinados por las relaciones entre todos los pares de tokens de entrada.

#### Configuración

Para este cuaderno, usaremos las siguientes librerías:

* [`torch`](https://pytorch.org/): La librería principal para construir y entrenar modelos de redes neuronales en este proyecto, incluyendo la implementación de mecanismos de *Self-Attention* (atención automática) y codificaciones posicionales (*Positional Encodings*).

* [`torch.nn`](https://pytorch.org/docs/stable/nn.html), [`torch.nn.functional`](https://pytorch.org/docs/stable/nn.functional.html): Estos submódulos de PyTorch se utilizan para definir las capas de la red neuronal y aplicar funciones como activaciones, que son esenciales en la construcción de la arquitectura del modelo.

* [`Levenshtein`](https://pypi.org/project/python-Levenshtein/): Esta librería se utiliza para calcular la distancia de Levenshtein, lo cual puede ser útil para evaluar el rendimiento del modelo en tareas como generación o traducción de texto, midiendo la diferencia entre las secuencias de texto predichas y las reales.

* [`get_tokenizer`](https://pytorch.org/text/stable/data_utils.html), [`build_vocab_from_iterator`](https://pytorch.org/text/stable/vocab.html) del módulo `torchtext`: Estas funciones son fundamentales para el preprocesamiento de datos de texto, incluyendo la tokenización del texto en palabras o subpalabras y la construcción de un vocabulario a partir del conjunto de datos, lo cual es un paso esencial en la preparación de datos para modelos de NLP (procesamiento de lenguaje natural).



#### Instalando las librerías requeridas


In [ ]:
# Instalando paquetes
#! pip install Levenshtein
#! pip install matplotlib
#!pip install torch==2.3.0 torchtext==0.18.0

#### Importando librerías requeridas


In [ ]:
import os
import sys
import time
import warnings
from pathlib import Path
import matplotlib.pyplot as plt

import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import requests

from Levenshtein import distance
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator

# También puedes usar esta sección para suprimir las advertencias generadas por tu código.
def warn(*args, **kwargs):
    pass
import warnings
warnings.warn = warn
warnings.filterwarnings('ignore')

En esta sección, inicializamos el entorno de entrenamiento de nuestra red neuronal y configuramos varios hiperparámetros para el modelo:

* **Configuración del dispositivo**: Asignamos los cálculos a una GPU si está disponible; de lo contrario, usamos la CPU. Utilizar una GPU puede acelerar significativamente el entrenamiento de modelos de aprendizaje profundo. Si CUDA está disponible, lo configuramos como `cuda`; de lo contrario, como `cpu`. *Compute Unified Device Architecture* (CUDA) es una plataforma de computación paralela y una interfaz de programación de aplicaciones (API) que permite al software aprovechar unidades de procesamiento gráfico (GPU) específicas para el procesamiento general acelerado, conocido como *General-Purpose Computing  on GPU* (GPGPU).

  > En este entorno de cuaderno, no se dispone de `cuda`.

* **Parámetros de entrenamiento**:

  * `learning_rate`: Es el tamaño del paso en cada iteración al acercarse a un mínimo de la función de pérdida. Lo fijamos en `3e-4`, un valor común de inicio para muchos modelos.
  * `batch_size`: Es el número de muestras que se propagan a través de la red en una sola pasada hacia adelante y hacia atrás. En este caso, es `64`.
  * `max_iters`: El número total de iteraciones de entrenamiento que planeamos ejecutar. Se establece en `5000` para darle al modelo una oportunidad amplia de aprender a partir de los datos.
  * `eval_interval` y `eval_iters`: Parámetros que definen con qué frecuencia evaluamos el rendimiento del modelo sobre un conjunto específico de lotes para aproximar la pérdida (*loss*).

* **Parámetros de arquitectura**:

  * `max_vocab_size`: Representa el número máximo de tokens en nuestro vocabulario. Se establece en `256`, lo que significa que solo consideraremos los 256 tokens más frecuentes.
  * `vocab_size`: Es el número real de tokens en el vocabulario, que puede ser menor al máximo debido a la longitud variable de los tokens en tokenizaciones subpalabra como BPE (Byte Pair Encoding).
  * `block_size`: La longitud de la secuencia de entrada que el modelo está diseñado para manejar. En este caso, es `16`.
  * `n_embd`: El tamaño de cada vector de *embedding* establecido en `32`. Los embeddings convierten los tokens en un espacio continuo donde tokens similares están más cerca entre sí.
  * `num_heads`: El número de cabeceras en el mecanismo de autoatención multi-cabecera (*multi-headed self-attention*), en este caso `2`, lo cual permite que el modelo atienda conjuntamente a información desde diferentes subespacios de representación.
  * `n_layer`: El número de capas (o profundidad) de la red. Aquí se utilizan `2` capas.
  * `ff_scale_factor`: Un factor de escala para el tamaño de las redes *feed-forward* (de avance), establecido en `4`.
  * `dropout`: La tasa de *dropout* utilizada como regularización para evitar el sobreajuste (*overfitting*), fijada en `0.0`, lo que indica que no se aplica *dropout* en este caso.

Finalmente, se incluye un cálculo de `head_size` derivado del tamaño de *embedding* y el número de cabeceras, asegurando que cada cabecera tenga una porción igual del tamaño del *embedding* para trabajar. También se incluye una aserción para verificar que `head_size` multiplicado por `num_heads` sea igual a `n_embd`.


In [ ]:
# Dispositivo para el entrenamiento
device = 'cuda' if torch.cuda.is_available() else 'cpu'
split = 'train'

# Parámetros de entrenamiento
learning_rate = 3e-4            # Tasa de aprendizaje
batch_size = 64                 # Tamaño del lote
max_iters = 5000                # Número máximo de iteraciones de entrenamiento
eval_interval = 200             # Evalua el modelo cada 'eval_interval' iteraciones durante el entrenamiento
eval_iters = 100                # Al evaluar, aproxima la pérdida usando 'eval_iters' lotes

# Parámetros de la arquitectura
max_vocab_size = 256            # Tamaño máximo del vocabulario
vocab_size = max_vocab_size     # Tamaño real del vocabulario (por ejemplo, BPE puede generar menos tokens que el máximo)
block_size = 16                 # Longitud del contexto para las predicciones
n_embd = 32                     # Tamaño del vector de embedding
num_heads = 2                   # Número de cabeceras en la atención multi-cabecera
n_layer = 2                     # Número de bloques (capas)
ff_scale_factor = 4             # Factor de escala para la red feed-forward (según el paper: d_model=512, d_ff=2048)
dropout = 0.0                   # Tasa de dropout para regularización (aquí, sin dropout)

# Tamaño de cada cabecera de atención
head_size = n_embd // num_heads
assert (num_heads * head_size) == n_embd  # Asegura que el embedding se divida equitativamente entre las cabeceras


Después de configurar los parámetros, crearás una función llamada `plot_embeddings`, la cual está diseñada para visualizar las *embeddings* aprendidas en un espacio 3D utilizando `matplotlib`. Esto ayuda a comprender cómo se agrupan y separan los diferentes *tokens*, proporcionando una visión sobre lo que el modelo ha aprendido.



In [ ]:
def plot_embdings(mi_embdings, name, vocab):

  fig = plt.figure()
  ax = fig.add_subplot(111, projection='3d')

  # Grafica los puntos de datos
  ax.scatter(mi_embdings[:,0], mi_embdings[:,1], mi_embdings[:,2])

  # Etiqueta los puntos
  for j, label in enumerate(name):
      i = vocab.get_stoi()[label]
      ax.text(mi_embdings[j,0], mi_embdings[j,1], mi_embdings[j,2], label)

  # Establece etiquetas para los ejes
  ax.set_xlabel('Etiqueta X')
  ax.set_ylabel('Etiqueta Y')
  ax.set_zlabel('Etiqueta Z')

  # Muestra el gráfico
  plt.show()


### Programa para traducción literal

En la siguiente parte, exploraremos los conceptos fundamentales de la tokenización y la traducción mediante un programa sencillo para realizar una traducción literal del francés al inglés:

* Se define un `dictionary` que asigna palabras en francés a sus equivalentes en inglés, formando la base de la lógica de traducción.



In [ ]:
dictionary = {
    'le': 'the'
    , 'chat': 'cat'
    , 'est': 'is'
    , 'sous': 'under'
    , 'la': 'the'
    , 'table': 'table'
}

* La función `tokenize` se encarga de dividir una oración en palabras individuales.
* La función `translate` utiliza esta función `tokenize` para dividir la oración de entrada y luego traduce cada palabra según el diccionario. Las palabras traducidas se concatenan para formar la oración de salida.

In [ ]:
# Función para dividir una oración en tokens (palabras)
def tokenize(text):
    """
    Esta función toma una cadena de texto como entrada y devuelve una lista de palabras (tokens).
    Utiliza el método split, que por defecto divide por cualquier espacio en blanco, para tokenizar el texto.
    """
    return text.split()  # Divide el texto de entrada por espacios en blanco y devuelve la lista de tokens

# Función para traducir una oración de un idioma origen a uno destino palabra por palabra
def translate(sentence):
    """
    Esta función traduce una oración buscando la traducción de cada palabra en un diccionario predefinido.
    Se asume que cada palabra de la oración es una clave en el diccionario.
    """
    out = ''  # Inicializa la cadena de salida
    for token in tokenize(sentence):  # Tokeniza la oración en palabras
        # Añade la palabra traducida a la cadena de salida
        # Esta línea asume que el diccionario contiene una traducción para cada palabra de entrada
        out += dictionary[token] + ' '
    return out.strip()  # Devuelve la oración traducida, eliminando espacios en blanco sobrantes



* Finalmente, se muestra el uso de la función `translate` con la entrada `"le chat est sous la table"`, que se traduce como `"the cat is under the table"` en inglés.



In [ ]:
translate("le chat est sous la table")

Este ejemplo sencillo ilustra una sustitución palabra por palabra que, aunque no es sofisticada, ofrece una introducción a los métodos de traducción computacional.



### Mejora: ¿Qué pasa si la 'clave' no está en el diccionario?

El código presenta una mejora al programa de traducción, abordando el caso en que una palabra no existe en el diccionario:

* **Función `find_closest_key`**: Esta nueva función tiene como objetivo encontrar la clave más cercana en el diccionario a una palabra dada. Utiliza la **distancia de Levenshtein** para encontrar la clave del diccionario con la distancia mínima a la palabra de consulta, sugiriendo una palabra similar si no se encuentra una coincidencia exacta.

* **Función `translate` mejorada**: La función `translate` se actualiza para utilizar `find_closest_key`. Ahora, en lugar de traducir directamente los *tokens* usando el diccionario, primero encuentra la clave más cercana para cada palabra tokenizada. Esto permite una traducción más robusta, especialmente cuando se encuentran palabras con errores ortográficos menores o variaciones que no están presentes en el diccionario.

* **Demostración**: Se demuestra la función `translate` mejorada con la entrada `"tables"`. Aunque `"tables"` no está en el diccionario, se espera que la función encuentre y utilice la clave más cercana `"table"` para la traducción, produciendo como salida `"table"` en inglés.

Esta mejora muestra una forma simple de manejo de errores y coincidencia difusa (*fuzzy matching*) en sistemas de traducción, permitiendo traducciones más flexibles y tolerantes a fallos.

In [ ]:
# Función para encontrar la clave más cercana en el diccionario a una palabra dada
def find_closest_key(query):
    """
    La función calcula la distancia de Levenshtein entre la palabra de consulta y cada clave en el diccionario.
    La distancia de Levenshtein mide el número de ediciones de un solo carácter necesarias para transformar una palabra en otra.
    """
    closest_key, min_dist = None, float('inf')  # Inicializa la clave más cercana y la distancia mínima al infinito
    for key in dictionary.keys():
        dist = distance(query, key)  # Calcula la distancia de Levenshtein con la clave actual
        if dist < min_dist:  # Si la distancia actual es menor que la mínima encontrada previamente
            min_dist, closest_key = dist, key  # Actualiza la distancia mínima y la clave más cercana
    return closest_key  # Devuelve la clave más cercana encontrada

# Función para traducir una oración del idioma origen al destino usando el diccionario
def translate(sentence):
    """
    Esta función tokeniza la oración de entrada en palabras y encuentra la traducción más cercana para cada palabra.
    Construye la oración traducida concatenando las palabras traducidas.
    """
    out = ''  # Inicializa la cadena de salida
    for query in tokenize(sentence):  # Tokeniza la oración en palabras
        key = find_closest_key(query)  # Encuentra la clave más cercana en el diccionario para cada palabra
        out += dictionary[key] + ' '  # Añade la traducción de la clave más cercana a la cadena de salida
    return out.strip()  # Devuelve la oración traducida, eliminando espacios en blanco sobrantes

In [ ]:
translate("tables")

#### Convierte a una red neuronal

Al pasar de una traducción básica a redes neuronales, comenzamos definiendo los vocabularios de entrada y salida y luego codificando los *tokens*:

* **Definición de vocabularios**: Se crean dos vocabularios a partir del diccionario, `vocabulary_in` para el idioma origen (francés) y `vocabulary_out` para el idioma destino (inglés). Estos vocabularios son listas de palabras únicas obtenidas de las claves y valores del diccionario, respectivamente, y se ordenan para mantener un orden consistente.

* **Codificación one-hot**: Se introduce la función `encode_one_hot` para convertir cada palabra del vocabulario en un vector codificado *one-hot*. La codificación *one-hot* es un proceso que representa cada palabra como un vector binario con un `1` en la posición correspondiente al índice de la palabra en el vocabulario y `0` en las demás posiciones. Esto crea un vector único de tamaño fijo para cada palabra, lo cual es esencial para el procesamiento en redes neuronales.

* **Demostración de codificación**: Se demuestra el proceso de codificación *one-hot* aplicando `encode_one_hot` a nuestro vocabulario de entrada (`vocabulary_in`) y mostrando los vectores codificados para cada palabra. Luego, se aplica el mismo proceso al vocabulario de salida (`vocabulary_out`).

Este paso es crucial en el aprendizaje automático, ya que prepara nuestros datos textuales para ser utilizados como entrada en una red neuronal, permitiéndole aprender y hacer predicciones a partir de esos datos.


#### Se define  los'vocabularios'


In [ ]:
# Crea y ordena el vocabulario de entrada a partir de las claves del diccionario
vocabulary_in = sorted(list(set(dictionary.keys())))
# Muestra el tamaño y el vocabulario ordenado para el idioma de entrada
print(f"Entrada del vocabulario ({len(vocabulary_in)}): {vocabulary_in}")

# Crea y ordena el vocabulario de salida a partir de los valores del diccionario
vocabulary_out = sorted(list(set(dictionary.values())))
# Muestra el tamaño y el vocabulario ordenado para el idioma de salida
print(f"Salida del vocabulario  ({len(vocabulary_out)}): {vocabulary_out}")

#### Codificar tokens usando codificación 'one-hot'

In [ ]:
# Función para convertir una lista de palabras del vocabulario en vectores codificados one-hot
def encode_one_hot(vocabulary):
    vocabulary_size = len(vocabulary)  # Obtiene el tamaño del vocabulario
    one_hot = dict()  # Inicializa un diccionario para almacenar las codificaciones one-hot
    LEN = len(vocabulary)  # La longitud de cada vector one-hot será igual al tamaño del vocabulario
    
    # Itera sobre el vocabulario para crear un vector codificado one-hot para cada palabra
    for i, key in enumerate(vocabulary):
        one_hot_vector = torch.zeros(LEN)  # Inicia con un vector de ceros
        one_hot_vector[i] = 1  # Establece en 1 la posición i para la palabra actual
        one_hot[key] = one_hot_vector  # Asocia la palabra con su vector codificado
        print(f"{key}\t: {one_hot[key]}")  # Imprime cada palabra y su vector codificado
    
    return one_hot  # Devuelve el diccionario de palabras y sus vectores one-hot

In [ ]:
# Aplica la función de codificación one-hot al vocabulario de entrada y almacenar el resultado
one_hot_in = encode_one_hot(vocabulary_in)

In [ ]:
# Itera sobre el vocabulario de entrada codificado one-hot y muestra cada vector
# Esto visualiza la representación one-hot de cada palabra del vocabulario de entrada
for k, v in one_hot_in.items():
    print(f"E_{{ {k} }} = " , v)

In [ ]:
# Aplica la función de codificación one-hot al vocabulario de salida y almacena el resultado
# En este caso se codifica el vocabulario del idioma destino
one_hot_out = encode_one_hot(vocabulary_out)

#### Vamos a crear un 'diccionario' usando multiplicación de matrices

Ahora ilustramos cómo crear una representación del diccionario adecuada para operaciones en redes neuronales:

* **Creación de matrices**: Usando `torch.stack` de PyTorch, convertimos los vectores codificados *one-hot* de los vocabularios de entrada (`K`) y salida (`V`) en tensores. `K` se construye a partir de los vectores *one-hot* del vocabulario de entrada, y `V` a partir de los vectores del vocabulario de salida. Estos tensores pueden considerarse como una tabla de consulta que el modelo usará para asociar tokens de entrada con tokens de salida.

* **Diccionario como matrices**: Este paso traduce efectivamente el mapeo de palabra a palabra en un formato amigable para redes neuronales. Cada fila en `K` corresponde a una palabra del idioma de entrada representada como un vector *one-hot*, y cada fila en `V` corresponde a la palabra traducida respectiva en el idioma de salida.

* **Ejemplo de consulta**: Se muestra un ejemplo de cómo usar operaciones de matrices para encontrar una traducción. Se consulta el vector *one-hot* de la palabra `"sous"` del vocabulario de entrada (`q`). Luego se demuestra cómo encontrar su traducción correspondiente mediante la multiplicación matricial con la transpuesta de `K` (es decir, `q @ K.T`) para identificar el índice y luego usar ese índice para seleccionar la fila relevante de `V`. Este proceso imita la búsqueda que realizaría una red neuronal real durante tareas de traducción.

Esta representación matricial es un paso previo para comprender cómo arquitecturas más complejas de redes neuronales, como aquellas que usan atención automática (*self-attention*), manejan las traducciones de tokens.


In [ ]:
# Apilamiento de los vectores codificados one-hot del vocabulario de entrada para formar un tensor
K = torch.stack([one_hot_in[k] for k in dictionary.keys()])
# K representa ahora una matriz de vectores one-hot para el vocabulario de entrada

# Mostrar el tensor para verificación
print(K)

In [ ]:
# De forma similar, apilar los vectores one-hot codificados del vocabulario de salida para formar un tensor
V = torch.stack([one_hot_out[k] for k in dictionary.values()])
# V representa la matriz correspondiente de vectores one-hot para el vocabulario de salida

# Mostrar el tensor para verificación
print(V)

In [ ]:
# Demostración de cómo consultar una traducción para una palabra dada usando operaciones matriciales
# Aquí, tomamos la representación one-hot de 'sous' del vocabulario de entrada
q = one_hot_in['sous']
# Mostrar el vector del token de consulta
print("Query :", q)

In [ ]:
# Selecciona el vector clave correspondiente en K (matriz del diccionario de entrada) usando multiplicación matricial
# Esta operación nos da el índice donde 'sous' tendría un 1 en la codificación one-hot del vocabulario de entrada
print("Key (K) :", q @ K.T)

In [ ]:
# Usa el índice encontrado desde la selección de clave para encontrar el vector valor correspondiente en V (matriz del diccionario de salida)
# Esta operación selecciona la fila de V que es la traducción de 'sous' en el vocabulario de salida
print("Value (V):", q @ K.T @ V)

# El resultado final demuestra cómo 'sous' puede ser traducido usando el enfoque de red neuronal

**Vector de consulta q, matriz K y matriz V:**

$$
q = \left[\begin{matrix}
  0 & 0 & 0 & 0 & 1 & 0\\\\\\\\\\\\
\end{matrix}\right]
, \
K = \left[\begin{matrix}
  0 & 0 & 0 & 1 & 0 & 0\\\\
  1 & 0 & 0 & 0 & 0 & 0\\\\
  0 & 1 & 0 & 0 & 0 & 0\\\\
  0 & 0 & 0 & 0 & 1 & 0\\\\
  0 & 0 & 1 & 0 & 0 & 0\\\\
  0 & 0 & 0 & 0 & 0 & 1\\\\
\end{matrix}\right]
 \
V = \left[\begin{matrix}
  0 & 0 & 0 & 1 & 0\\\\
  1 & 0 & 0 & 0 & 0\\\\
  0 & 1 & 0 & 0 & 0\\\\
  0 & 0 & 0 & 0 & 1\\\\
  0 & 0 & 0 & 1 & 0\\\\
  0 & 0 & 1 & 0 & 0\\\\
\end{matrix}\right]
$$


La operación $q \cdot K^T \cdot V$ nos permite construir una estructura tipo diccionario a partir de un conjunto de vectores.

Este es un ejemplo de cómo seleccionar el valor desde una consulta:


$$
q \cdot K^T \cdot V =
\left[\begin{matrix}
  0 & 0 & 0 & 0 & 1 & 0\\\\\\\\\\\\
\end{matrix}\right]
\cdot
\left[\begin{matrix}
  0 & 1 & 0 & 0 & 0 & 0\\\\
  0 & 0 & 1 & 0 & 0 & 0\\\\
  0 & 0 & 0 & 0 & 1 & 0\\\\
  1 & 0 & 0 & 0 & 0 & 0\\\\
  0 & 0 & 0 & 1 & 0 & 0\\\\
  0 & 0 & 0 & 0 & 0 & 1\\\\
\end{matrix}\right]
\cdot
\left[\begin{matrix}
  0 & 0 & 0 & 1 & 0\\\\
  1 & 0 & 0 & 0 & 0\\\\
  0 & 1 & 0 & 0 & 0\\\\
  0 & 0 & 0 & 0 & 1\\\\
  0 & 0 & 0 & 1 & 0\\\\
  0 & 0 & 1 & 0 & 0\\\\
\end{matrix}\right]
$$


$$
q \cdot K^T \cdot V =
%\hspace{2cm}
\left[\begin{matrix}
  0 & 0 & 0 & 1 & 0 & 0\\\\\\\\\\\\
\end{matrix}\right]
%\hspace{2.5cm}
\cdot
\left[\begin{matrix}
  0 & 0 & 0 & 1 & 0\\\\
  1 & 0 & 0 & 0 & 0\\\\
  0 & 1 & 0 & 0 & 0\\\\
  0 & 0 & 0 & 0 & 1\\\\
  0 & 0 & 0 & 1 & 0\\\\
  0 & 0 & 1 & 0 & 0\\\\
\end{matrix}\right]
\hspace{4.5cm}
$$


$$
q \cdot K^T \cdot V
=
%\hspace{3.5cm}
\left[\begin{matrix}
0 & 0 & 0 & 0 & 1\\\\\\\\\\\\
\end{matrix}\right]
%\hspace{3.5cm}
\hspace{9cm}
$$


El código introduce una función para decodificar vectores *one-hot* a tokens y actualiza la función de traducción para utilizar multiplicación de matrices:

#### Decodificar vector *one-hot*

La función `decode_one_hot` está diseñada para decodificar un vector codificado *one-hot* y devolver el token correspondiente. Lo hace encontrando el token cuya representación *one-hot* tiene la mayor similitud del coseno con el vector dado, que en este caso se reduce al producto punto, dado que los vectores *one-hot* son ortogonales.


In [ ]:
def decode_one_hot(one_hot, vector):
    """ 
    Decodifica un vector one-hot para encontrar el token que mejor coincide en el vocabulario.
    """
    best_key, best_cosine_sim = None, 0
    for k, v in one_hot.items():  # Itera sobre el vocabulario codificado one-hot
        cosine_sim = torch.dot(vector, v)  # Calcula el producto punto (similitud del coseno)
        if cosine_sim > best_cosine_sim:  # Si es la mejor similitud encontrada
            best_cosine_sim, best_key = cosine_sim, k  # Actualiza la mejor similitud y el token correspondiente
    return best_key  # Devuelve el token correspondiente al vector one-hot

#### Función de traducción basada en matrices

La función `translate` ahora utiliza operaciones matriciales para realizar la traducción. Para cada token en la oración de entrada, encuentra su vector *one-hot*, lo multiplica con las matrices `K.T` y `V` para obtener el vector codificado en el vocabulario de salida, y luego decodifica este vector para obtener la palabra traducida.

In [ ]:
def translate(sentence):
    """ 
    Traduce una oración usando multiplicación de matrices, tratando los diccionarios como matrices.
    """
    sentence_out = ''  # Inicializa la oración de salida
    for token_in in tokenize(sentence):  # Tokeniza la oración de entrada
        q = one_hot_in[token_in]  # Encuentra el vector one-hot del token
        out = q @ K.T @ V  # Multiplica con las matrices de entrada y salida para obtener la traducción
        token_out = decode_one_hot(one_hot_out, out)  # Decodifica el vector one-hot de salida a un token
        sentence_out += token_out + ' '  # Añade el token traducido a la oración de salida
    return sentence_out.strip()  # Devuelve la oración traducida

#### Prueba de traducción

Se prueba la función mejorada `translate` con la oración `"le chat est sous la table"`, verificando que se traduzca correctamente como `"the cat is under the table"` utilizando operaciones matriciales para una traducción palabra por palabra sin fisuras.


In [ ]:
translate("le chat est sous la table")

Este enfoque mejorado muestra cómo los modelos de redes neuronales pueden traducir idiomas representando el diccionario de traducción como matrices y utilizando operaciones con vectores.


**El siguiente segmento de código introduce conceptos que conducen a la implementación de "Atención" en redes neuronales:**


#### Función softmax para similitud

Se explica que los tokens similares tendrán vectores similares, y se añade una función softmax a la ecuación. Esta función se aplica a la salida de la multiplicación matricial del vector de consulta `q` y la transpuesta de la matriz `K`. La función softmax convierte estos valores en probabilidades, destacando el token más similar aunque sin descartar completamente a los demás.


In [ ]:
print('E_{table} = ', one_hot_in['table'])

$$
E_{table} =  \left[\begin{matrix}
  0 & 0 & 0 & 0 & 0 & 1\\\\\\\\\\\\
\end{matrix}\right]
\ \ \
$$

$$
E_{tables} =  \left[\begin{matrix}
  0 & 0 & 0 & 0 & 0 & 0.95\\\\
\end{matrix}\right]
$$


La nueva ecuaciones:
$$
softmax(q \cdot K^T) \cdot V
$$

Y ajustándola según la dimensionalidad del vector de consulta, obtenemos:

$$
softmax\left( \frac{q \cdot K^T}{\sqrt{d}} \right) \cdot V
$$


#### Traducción con mecanismo de atención

La función `translate` se modifica para usar la función softmax como forma de aplicar atención. Primero obtiene el vector one-hot del token, luego aplica la función softmax al producto punto de `q` y `K.T`, lo escala por la raíz cuadrada de la dimensión (para propósitos de normalización), y finalmente lo multiplica por `V` para obtener el vector de salida.

In [ ]:
def translate(sentence):
    """
    Traduce una oración usando el mecanismo de atención representado por las matrices K y V.
    Se usa la función softmax para calcular una suma ponderada de los vectores de V, 
    enfocándose en el vector más relevante para la traducción.
    """
    sentence_out = ''  # Inicializa la oración de salida
    for token_in in tokenize(sentence):  # Tokeniza la oración de entrada
        q = one_hot_in[token_in]  # Obtiene el vector one-hot del token actual
        # Aplica softmax al producto punto escalado de q y K.T, luego multiplicar por V
        # Esto selecciona el vector de traducción más relevante desde V
        out = torch.softmax(q @ K.T, dim=0) @ V
        token_out = decode_one_hot(one_hot_out, out)  # Decodifica el vector de salida a un token
        sentence_out += token_out + ' '  # Añade el token traducido a la oración de salida
    return sentence_out.strip()  # Devuelve la oración traducida

# Prueba la función de traducción
translate("le chat est sous la table")

**Prueba de traducción**: Se prueba la función `translate` actualizada para asegurar que procesa correctamente la oración de ejemplo `"le chat est sous la table"`, traduciéndola como `"the cat is under the table"`. Esto verifica que el mecanismo de atención implementado usando softmax funciona correctamente.

Este paso marca la transición de una traducción basada en búsqueda directa a un enfoque basado en atención, introduciendo un componente clave de los modelos modernos de traducción neuronal.



**La siguiente parte del código demuestra una mejora en el proceso de traducción al manejar todas las consultas en paralelo:**


#### Creación de la matriz 'Q'

La matriz `Q` se construye apilando los vectores codificados one-hot de todos los tokens de la oración de entrada. Esto paraleliza el proceso de preparación de los vectores de consulta, siendo más eficiente que hacerlo de forma secuencial.


In [ ]:
# La oración que queremos traducir
sentence = "le chat est sous la table"

# Apila todos los vectores one-hot codificados para los tokens de la oración y formar la matriz Q
Q = torch.stack([one_hot_in[token] for token in tokenize(sentence)])

# Muestra la matriz Q
print(Q)

$$
Q = \left[\begin{matrix}
  0 & 0 & 0 & 1 & 0 & 0 \\\\\\\\\\\\
  1 & 0 & 0 & 0 & 0 & 0 \\\\
  0 & 1 & 0 & 0 & 0 & 0 \\\\
  0 & 0 & 0 & 0 & 1 & 0 \\\\
  0 & 0 & 1 & 0 & 0 & 0 \\\\
  0 & 0 & 0 & 0 & 0 & 1 \\\\
\end{matrix}\right]
$$


$$
Atención(Q, K, V) = softmax\left( \frac{Q \cdot K^T}{\sqrt{d}} \right) V
$$


#### Función `translate` actualizada

La función `translate` se revisa para utilizar multiplicación matricial sobre toda la oración. En lugar de traducir palabra por palabra, ahora usa la matriz `Q` para realizar la operación en paralelo para todas las palabras.


In [ ]:
def translate(sentence):
    """
    Traduce una oración usando multiplicación de matrices en paralelo.
    Esta función reemplaza el enfoque iterativo por un solo paso de multiplicación matricial,
    aplicando el mecanismo de atención a todos los tokens a la vez.
    """
    # Tokeniza la oración y apilar los vectores one-hot para formar la matriz Q
    Q = torch.stack([one_hot_in[token] for token in tokenize(sentence)])
    
    # Aplica softmax al producto punto de Q y K.T y multiplicar por V
    # Esto nos dará los vectores de salida para todos los tokens en paralelo
    out = torch.softmax(Q @ K.T, 0) @ V
    
    # Decodifica cada vector one-hot en la salida al token correspondiente
    # Y une los tokens para formar la oración traducida
    return ' '.join([decode_one_hot(one_hot_out, o) for o in out])
    
# Prueba la función para asegurar que produce la traducción correcta
translate("le chat est sous la table")

* **Mejora en eficiencia**: Al aplicar operaciones a toda la oración de una vez, este enfoque simula un aspecto clave del mecanismo de atención real usado en redes neuronales: procesar múltiples componentes de entrada en paralelo para una computación más rápida.

* **Salida de prueba**: La función actualizada traduce correctamente la oración en francés `"le chat est sous la table"` como `"the cat is under the table"`, confirmando que la paralelización funciona de manera efectiva.

Esta optimización apunta a las ventajas computacionales de las operaciones matriciales en redes neuronales, particularmente en tareas como la traducción, que se benefician del procesamiento paralelo.


#### Clase de self-attention (opcional)

En esta sección, aprenderemos cómo crear cabeceras de atención automática (*self-attention heads*) desde cero. Este es un componente opcional del cuaderno, ya que también puedes crear cabeceras de self-attention utilizando los modelos `transformer` de PyTorch.

La clase `Head` representa una cabecera de self-attention. Hereda de `nn.Module` de PyTorch, lo que la convierte en parte de una red neuronal que puede aprender de los datos.

<p style="text-align:center">
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMSkillsNetwork-AI0201EN-Coursera/attention.png" width="200" alt="attention"/>
</p>


$$
Atención(Q, K, V)  => Atención(Q \cdot W^Q, K \cdot W^K, V \cdot W^V)
$$


In [ ]:
class Head(nn.Module):
    """ Cabecera de self-attention. Esta clase implementa un mecanismo de atención automática
        que es un componente clave de las arquitecturas de redes neuronales tipo transformer. """

    def __init__(self):
        super().__init__()  # Inicializa la superclase (nn.Module)
        # Capa de embedding para convertir índices de tokens en vectores de tamaño fijo (n_embd)
        self.embedding = nn.Embedding(vocab_size, n_embd)
        # Capas lineales para calcular las consultas (query), claves (key) y valores (value) a partir de los embeddings
        self.key = nn.Linear(n_embd, n_embd, bias=False)
        self.query = nn.Linear(n_embd, n_embd, bias=False)
        self.value = nn.Linear(n_embd, n_embd, bias=False)

    def attention(self, x):
        embedded_x = self.embedding(x)
        k = self.key(embedded_x)
        q = self.query(embedded_x)
        v = self.value(embedded_x)
        # Ponderación de atención
        w = q @ k.transpose(-2, -1) * k.shape[-1]**-0.5   # Consulta * Claves / normalización
        w = F.softmax(w, dim=-1)  # Aplica softmax a lo largo de la última dimensión
        return embedded_x, k, q, v, w
    
    def forward(self, x):
        embedded_x = self.embedding(x)
        k = self.key(embedded_x)
        q = self.query(embedded_x)
        v = self.value(embedded_x)
        # Ponderación de atención
        w = q @ k.transpose(-2, -1) * k.shape[-1]**-0.5   # Consulta * Claves / normalización
        w = F.softmax(w, dim=-1)  # Aplica softmax a lo largo de la última dimensión
        # Añade los valores ponderados
        out = w @ v
        return out

#### Definición del conjunto de datos

Para ilustrar las transformaciones sobre texto de entrada, se define un conjunto de datos de ejemplo, que parece ser una lista de tuplas, cada una con un ID y una cadena de texto relacionada con tareas de NLP.


In [ ]:
dataset = [
    (1,"Introduction to NLP"),
    (2,"Basics of PyTorch"),
    (1,"NLP Techniques for Text Classification"),
    (3,"Named Entity Recognition with PyTorch"),
    (3,"Sentiment Analysis using PyTorch"),
    (3,"Machine Translation with PyTorch"),
    (1," NLP Named Entity,Sentiment Analysis,Machine Translation "),
    (1," Machine Translation with NLP "),
    (1," Named Entity vs Sentiment Analysis  NLP "),
    (3,"he painted the car red"),
    (1,"he painted the red car")
    ]

#### Configuración del tokenizador

Se crea un tokenizador utilizando la función `get_tokenizer` de `torchtext`, que se encargará de dividir cadenas en tokens (palabras).

In [ ]:
tokenizer = get_tokenizer("basic_english")  # Obtiene un tokenizador básico en inglés

#### Construcción del vocabulario

La función `yield_tokens` itera sobre el conjunto de datos y produce versiones tokenizadas del texto. Estos tokens luego se usan con `build_vocab_from_iterator` para crear un objeto de vocabulario, que incluye un token especial `<unk>` para palabras desconocidas.



In [ ]:
def yield_tokens(data_iter):
    """Genera listas de tokens a partir del conjunto de datos."""
    for _, text in data_iter:
        yield tokenizer(text)

vocab = build_vocab_from_iterator(yield_tokens(dataset), specials=["<unk>"])
vocab.set_default_index(vocab["<unk>"])  # Establece índice por defecto para palabras desconocidas

#### Pipeline de procesamiento de texto

Se define una función `text_pipeline` para convertir cadenas de texto crudas en índices de tokens usando el tokenizador y el vocabulario.

In [ ]:
def text_pipeline(x):
    """Convierte una cadena de texto en una lista de índices de tokens."""
    return vocab(tokenizer(x))  # Tokeniza la entrada y mapear cada token a su índice en el vocabulario

#### Definición de hiperparámetros

Se especifican hiperparámetros para el modelo, incluyendo el tamaño del vocabulario y la dimensión del espacio de embedding.


In [ ]:
vocab_size = len(vocab)  # Número total de tokens en el vocabulario
n_embd = 3  # Dimensión del espacio de embedding

# Crea la cabecera de atención con la capa de embedding integrada
attention_head = Head()

#### Datos de prueba

Se crean datos de entrada simulados para pruebas. Se aplica `text_pipeline` para convertir el texto en un tensor de índices de tokens, que será usado como entrada para el modelo de red neuronal.

In [ ]:
# Define la oración a tokenizar y convertir a índices
mi_tokens = 'he painted the car red'
# Aplica el pipeline de texto para obtener los índices de tokens
input_data = torch.tensor(text_pipeline(mi_tokens), dtype=torch.long)

# Imprime la forma y el tensor de índices de tokens
print(input_data.shape)
print(input_data)

In [ ]:
# Pasa los datos tokenizados por la capa de embedding y el mecanismo de atención de la clase Head
embedded_x, k, q, v, w = attention_head.attention(input_data)

# Imprime el tamaño del vector embeddeado resultante para verificar
print(embedded_x.shape)  # Debería mostrar la forma [número de tokens, dimensión de embedding]
print("embedded_x:", embedded_x)  # Las representaciones embeddeadas reales de los tokens de entrada

In [ ]:
# Imprime las formas de las matrices de clave, consulta, valor y pesos de atención
# Esto ayuda a verificar que las dimensiones sean correctas para los cálculos de atención
print("k:", k.shape)  # Forma del tensor de claves
print("q:", q.shape)  # Forma del tensor de consultas
print("v:", v.shape)  # Forma del tensor de valores
print("w:", w.shape)  # Forma del tensor de pesos de atención

In [ ]:
# Ahora pasamos los datos de entrada por toda la cabecera de atención para obtener la salida
output = attention_head(input_data)

# Imprime la salida y su forma, que debe coincidir con la forma de entrada
# El tensor de salida contiene las representaciones resultantes después de aplicar la atención
print(output.shape)
print(output)

###  **Codificación posicional**

Para ilustrar, considera las oraciones: *'He painted the car red'* y *'He painted the red car'*. Estas oraciones tienen significados diferentes debido a la posición distinta de las palabras *'red'* y *'car'*. Una codificación posicional bien implementada permitiría al modelo distinguir entre las posiciones de estas palabras, asegurando que los embeddings de estas oraciones sean distintos.

En tareas de procesamiento de secuencias, es crucial que el modelo entienda la posición de los elementos dentro de una secuencia. Esto se logra asociando a cada muestra un índice de posición único, denotado como $p_{n}=n$, donde $n$ representa la posición de la muestra en la secuencia. Por ejemplo, asignamos el índice 0 al primer elemento, el índice 1 al segundo, y así sucesivamente. Esta técnica simple pero efectiva permite al modelo comprender las posiciones relativas de los elementos en la secuencia. 

Es importante destacar que este método se mantiene confiable incluso para secuencias largas, hasta donde lo permita nuestro vocabulario. Al incorporar codificaciones posicionales, nuestro modelo adquiere un entendimiento profundo del orden y la disposición de los elementos, mejorando su rendimiento en diversas tareas relacionadas con secuencias.

La posición que se imprime a continuación incorpora toda la longitud del tamaño del vocabulario.


In [ ]:
# Genera un rango de índices de posición desde 0 hasta el tamaño del vocabulario
position = torch.arange(0, vocab_size, dtype=torch.float).unsqueeze(1)

# Obtiene la lista de palabras desde el objeto vocabulario
vocab_list = list(vocab.get_itos())

# Itera sobre el rango del tamaño del vocabulario
for idx in range(vocab_size):
    word = vocab_list[idx]  # Obtiene la palabra del vocabulario en el índice actual
    pos = position[idx][0].item()  # Extrae el valor numérico del índice de posición desde el tensor
    print(f"Palabra: {word}, Índice de posición: {pos}")

En codificaciones posicionales, cada dimensión $d$ dentro del embedding tiene importancia específica, denotada como $\textbf{p}_{n,d}$. Por ejemplo, si tienes una dimensión de embedding de tres, las codificaciones posicionales $\textbf{p}_{n,d}$ se representan como $[\text{ }p_{n,1},\text{ }p_{n,2},\text{ }p_{n,3}]$. Estos valores corresponden a un embedding tridimensional para cada token, capturando su posición y características únicas dentro de la secuencia. Al organizar la información posicional de esta forma, el modelo gana un entendimiento mejor de la relación entre los tokens.



In [ ]:
# Inicializa una matriz de ceros con dimensiones [vocab_size, n_embd]
# Esto se usará para contener las codificaciones posicionales para cada palabra en el vocabulario
pe = torch.zeros(vocab_size, n_embd)

# Concatena el tensor de posiciones tres veces a lo largo de la dimensión 1 (columnas)
# Este ejemplo simple probablemente sea un marcador de posición para una función más compleja en una codificación posicional real
pe = torch.cat((position, position, position), 1)

# Muestra el tensor de codificación posicional
# En una implementación real, esto implicaría un método más sofisticado que refleje el impacto de la posición en los embeddings
pe

Solo necesitas codificaciones posicionales para cada secuencia de embeddings. Para determinar esto, cuenta cuántos embeddings hay en la secuencia. Refiérete a la oración *'he painted the car red he painted the red car'* en **`mi_embdings`**.


In [ ]:
# Obtiene la forma del tensor embedded_x que contiene las representaciones embedidas
# 'samples' contendrá el número de tokens/muestras en el lote
# 'dim' contendrá la dimensión de los embeddings
samples, dim = embedded_x.shape

# Imprime la tupla (samples, dim) para mostrar el tamaño del lote y la dimensión del embedding
samples, dim

Agrega las codificaciones posicionales a los embeddings y grafica los resultados; observa que son diferentes.


In [ ]:
# Selecciona las codificaciones posicionales apropiadas basadas en el número de muestras y dimensiones
# Este corte del tensor de codificación posicional 'pe' corresponde al tamaño real del lote y del embedding
pe_slice = pe[0:samples, 0:n_embd]

In [ ]:
# Agrega las codificaciones posicionales a los embeddings de tokens
# Este paso integra la información de posición en los embeddings
pos_embding = embedded_x + pe_slice

# El resultado 'pos_embding' ahora contiene los embeddings de tokens ajustados con la información posicional
# Muestra el tensor de embeddings ajustados
pos_embding

In [ ]:
# Visualiza los embeddings con codificaciones posicionales usando la función definida de graficado
# 'pos_embding.detach().numpy()' convierte el tensor a un arreglo de NumPy y lo desconecta del grafo de cómputo
# 'tokenizer(my_tokens)' tokeniza la oración de ejemplo para el etiquetado en la gráfica
# 'vocab' se pasa para asociar tokens con sus embeddings durante la graficación
plot_embdings(pos_embding.detach().numpy(), tokenizer(mi_tokens), vocab)
plot_embdings(pos_embding.detach().numpy(),tokenizer(mi_tokens),vocab)

Al incorporar codificaciones posicionales lineales en los embeddings, surge un problema importante: los embeddings para posiciones posteriores tienden a volverse mucho más grandes. Esto se vuelve especialmente evidente con secuencias largas. Por ejemplo, considera la palabra *'car'* que aparece en los índices 3 y 9 dentro de una secuencia. El embedding de *'car'* en la posición 9 es visiblemente más grande que en la posición 3.

Esto es problemático porque los embeddings están diseñados para representar palabras o tokens de manera coherente. Las diferencias excesivas de magnitud pueden hacer que el modelo tenga dificultades para interpretar y procesar la secuencia. Es crucial abordar este problema para preservar la integridad de los embeddings y garantizar que la información posicional mejore y no distorsione la representación de los tokens.


In [ ]:
pos_embding[3]# agrega -3 para obtener el embedding origen

In [ ]:
pos_embding[-1]# agrega -9 para obtener el embedding original

Aborda este problema de magnitud usando diversas estrategias. Primero, emplea una función que no amplifique tanto los valores. Además, puedes introducir funciones distintas. Por ejemplo, las codificaciones posicionales $\textbf{p}_{n,d}$ pueden representarse como $[0.1n, -0.1n, 0]$.

En este esquema modificado, las magnitudes de las dos primeras dimensiones $[p_{n,1}, p_{n,2}]$ aumentan linealmente con la posición $n$ pero a un ritmo más lento (0.1 veces la posición). La tercera dimensión $[p_{n,3}]$ permanece constante en 0. Ajustando estos coeficientes, puedes controlar el ritmo de crecimiento, asegurando que los embeddings no escalen en exceso y proporcionando una representación más equilibrada.


In [ ]:
pe=torch.cat((0.1*position, -0.1*position, 0*position), 1)

Se grafica las codificaciones posicionales.


In [ ]:
# Grafica la primera dimensión de las codificaciones posicionales para todas las posiciones
plt.plot(pe[:, 0].numpy(), label="Dimensión 1")

# Grafica la segunda dimensión
plt.plot(pe[:, 1].numpy(), label="Dimensión 2")

# Grafica la tercera dimensión
plt.plot(pe[:, 2].numpy(), label="Dimensión 3")

# Etiqueta el eje x como 'Número de secuencia'
plt.xlabel("Número de secuencia")

# Agrega una leyenda para identificar cada dimensión
plt.legend()

plt.show()

Observa que las palabras están más cerca entre sí, pero el uso de una función lineal sigue presentando los mismos inconvenientes: eventualmente los embeddings seguirán creciendo linealmente.


In [ ]:
# Después de haber sumado las codificaciones posicionales, necesitamos separar el tensor del grafo computacional
# y convertirlo a un arreglo de NumPy para su visualización
# 'detach()' es necesario porque 'embedded_x' requiere gradientes y no los necesitamos para graficar
# 'numpy()' convierte el tensor de PyTorch a un arreglo de NumPy
pos_embding_numpy = pos_embding.detach().numpy()

# Tokeniza la oración de ejemplo para usar como etiquetas en la gráfica
tokens = tokenizer(mi_tokens)

# Visualiza los embeddings de los tokens con codificaciones posicionales
# Se asume que la función 'plot_embdings' recibe el arreglo NumPy de embeddings, etiquetas de tokens y un vocabulario
# para grafica los embeddings en un espacio donde los embeddings similares estén más cerca entre sí
plot_embdings(pos_embding_numpy, tokens, vocab)

Al analizar las codificaciones posicionales, podría parecer que las palabras están más cerca entre sí, lo que indica una mejora. Sin embargo, es importante reconocer que el uso de una función lineal aún conlleva sus propias limitaciones. A pesar de la impresión visual de cercanía, las codificaciones posicionales lineales tienen problemas inherentes. Uno de los más significativos es la amplificación potencial de magnitudes a medida que avanza la posición, lo que puede afectar negativamente la capacidad del modelo para captar matices posicionales sutiles.

Para superar estos desafíos, es valioso explorar métodos y funciones alternativas para generar codificaciones posicionales. Un enfoque prometedor consiste en utilizar funciones periódicas, como las funciones seno y coseno. Estas funciones poseen una propiedad única llamada **periodicidad**, lo que significa que repiten sus valores a intervalos regulares. Esta periodicidad evita que crezcan demasiado conforme aumenta la posición en la secuencia.

Al incorporar funciones seno y coseno en las codificaciones posicionales, se introduce un mecanismo más consciente del contexto y adaptativo. Estas funciones permiten capturar la información secuencial sin el riesgo de una escalada incontrolada de magnitudes. Como resultado, el modelo puede discernir mejor las posiciones relativas de las palabras dentro de la secuencia, lo que conduce a representaciones más precisas y significativas.

In [ ]:
# Genera codificaciones posicionales usando una función sinusoidal y concatenación
# La primera dimensión se codifica con una función seno
# La segunda y tercera dimensiones son marcadores de posición y simplemente se establecen en 1 (esto no es típico en la práctica y sirve como un ejemplo simplificado)
pe = torch.cat((torch.sin(2 * 3.14 * position / 6),  # Codificación sinusoidal para la dimensión 1
                0 * position + 1,                     # Codificación constante (1) para la dimensión 2
                0 * position + 1), axis=1)            # Codificación constante (1) para la dimensión 3

# Suma las codificaciones posicionales sinusoidales a los embeddings de los tokens
# Este paso enriquece los embeddings con información sobre la posición de cada token en la secuencia
pos_embding = embedded_x + pe[0:samples, :]

# Prepara los embeddings posicionales para visualización
# 'detach()' se usa para dejar de rastrear las operaciones sobre 'pos_embding'
# 'numpy()' convierte el tensor a un arreglo NumPy adecuado para graficar
pos_embding_numpy = pos_embding.detach().numpy()

# Tokeniza la oración de ejemplo para obtener las etiquetas para la gráfica de embeddings
tokens = tokenizer(mi_tokens)

# Visualiza los embeddings usando la función de graficado
# Se asume que esta función graficará los embeddings para ilustrar los efectos de las codificaciones posicionales
plot_embdings(pos_embding_numpy, tokens, vocab)


La primera dimensión de **$pe$** sigue un patrón de onda sinusoidal, mientras que la segunda y tercera dimensiones tienen valores constantes. Esto se muestra en la siguiente gráfica:

In [ ]:
pe

En la exploración de los *embeddings* de palabras, hemos notado un patrón fascinante: las palabras generalmente están posicionadas lo suficientemente cerca en el espacio de *embedding* para preservar su proximidad, asegurando que estén relacionadas contextualmente pero se mantengan distintas entre sí. Sin embargo, hay una excepción significativa que surge específicamente con la palabra *'car'*.

Este comportamiento peculiar puede atribuirse a la naturaleza de la onda sinusoidal utilizada en la codificación posicional. La onda seno es inherentemente periódica, lo que significa que repite su patrón a intervalos regulares. Esta periodicidad se ilustra visualmente en la gráfica, donde se observa la curva sinusoidal repitiéndose. Como resultado, la codificación posicional para la palabra *'car'* en diferentes ubicaciones dentro de la secuencia sigue siendo la misma.

Esta naturaleza periódica plantea un desafío, especialmente para palabras como *'car'*, que pueden aparecer varias veces en una secuencia pero tener significados contextuales diferentes cada vez. A pesar de sus distintos contextos, la codificación posicional de todas las apariciones de *'car'* sería idéntica debido al patrón repetitivo de la función seno.

In [ ]:
# Grafica las codificaciones posicionales con diferentes estilos de línea y marcadores
plt.plot(pe[:, 0].numpy(), label="Dimensión 1", linestyle='-')
plt.plot(pe[:, 1].numpy(), label="Dimensión 2", linestyle='--')
plt.plot(pe[:, 2].numpy(), label="Dimensión 3", linestyle=':')

# Ajusta la escala del eje y para una mejor visibilidad
plt.ylim([-1, 1.1])

plt.xlabel("Número de secuencia")
plt.legend()
plt.show()

Al incorporar funciones seno y coseno con diferentes frecuencias, se codifican secuencias de diferentes longitudes. Como se demuestra aquí, estas codificaciones posicionan los elementos de forma que mantienen su cercanía y, al mismo tiempo, conservan su individualidad.


In [ ]:
# Crea codificaciones posicionales usando funciones sinusoidales
# Concatena funciones coseno y seno para diferentes dimensiones para formar la codificación
pe = torch.cat((torch.cos(2 * 3.14 * position / 25),  # Función coseno para la primera dimensión
                torch.sin(2 * 3.14 * position / 25),  # Función seno para la segunda dimensión
                torch.sin(2 * 3.14 * position / 5)), axis=1)  # Seno con diferente frecuencia para la tercera dimensión

# Se agrega las codificaciones posicionales generadas a los embeddings de los tokens
# Esto enriquece los embeddings con información posicional
pos_embding = embedded_x + pe[0:samples, :]

# Separa los embeddings del grafo computacional y convertir a NumPy para visualización
pos_embding_numpy = pos_embding.detach().numpy()

# Tokeniza el texto de entrada para usar como etiquetas en la gráfica
tokens = tokenizer(mi_tokens)

# Visualiza los embeddings con codificaciones posicionales
plot_embdings(pos_embding_numpy, tokens, vocab)

# Además, grafica cada dimensión de las codificaciones posicionales para visualizar sus patrones
plt.plot(pe[:, 0].numpy(), label="Dimensión 1 - Onda coseno")
plt.plot(pe[:, 1].numpy(), label="Dimensión 2 - Onda seno")
plt.plot(pe[:, 2].numpy(), label="Dimensión 3 - Onda seno")

# Añade leyenda en la esquina superior izquierda fuera del área del gráfico
plt.legend(loc='upper left', bbox_to_anchor=(1, 1))

plt.show()

En general, las funciones periódicas ofrecen un medio más expresivo y adaptable para codificar información posicional en secuencias, por lo que son la opción preferida en modelos modernos de secuencia a secuencia como los *transformers*. Ofrecen capacidades mejoradas para modelar dependencias tanto de corto como de largo alcance, lo cual es crucial para tareas de procesamiento de lenguaje natural y otros dominios.


Ahora que se ha tratado la **codificación posicional**, **embeddings** y **self-attention**, unamos todas las capas para crear un modelo tipo *transformer* que reciba una secuencia y la transforme para su posterior procesamiento.

In [ ]:
class PositionalEncoding(nn.Module):
    """El módulo de codificación posicional inyecta información sobre la posición relativa o absoluta de los tokens en la secuencia."""
    def __init__(self, n_embd, vocab_size, dropout=0.1):
        super(PositionalEncoding, self).__init__()
        # Inicializa un búfer para las codificaciones posicionales (no es un parámetro, por lo tanto no se actualiza durante el entrenamiento)
        pe = torch.zeros(vocab_size, n_embd)
        position = torch.arange(0, vocab_size, dtype=torch.float).unsqueeze(1)
        # Calcula las codificaciones posicionales una sola vez en espacio logarítmico
        pe = torch.cat((torch.cos(2 * 3.14 * position / 25), torch.sin(2 * 3.14 * position / 25), torch.sin(2 * 3.14 * position / 5)), 1)
        self.register_buffer('pe', pe)

    def forward(self, x):
        # Suma la codificación posicional a cada vector de embedding asumiendo que x es (seq_len, batch_size, n_embd)
        # Nota: 'pe' es un búfer registrado y no requiere gradientes
        pos = x + self.pe[:x.size(0), :]
        return pos

class Head(nn.Module):
    """Cabecera de atención automática (self-attention)."""
    def __init__(self, n_embd, vocab_size):
        super().__init__()
        # Capa de embedding que convierte los índices de tokens en vectores densos de tamaño fijo
        self.embedding = nn.Embedding(vocab_size, n_embd)
        # Capa de codificación posicional
        self.pos_encoder = PositionalEncoding(n_embd, vocab_size)
        # Capas para transformar los embeddings codificados posicionalmente en queries, keys y values
        self.key = nn.Linear(n_embd, n_embd, bias=False)
        self.query = nn.Linear(n_embd, n_embd, bias=False)
        self.value = nn.Linear(n_embd, n_embd, bias=False)

    def forward(self, x):
        # Pasa la entrada por la capa de embedding para obtener vectores densos de tamaño fijo
        embedded_x = self.embedding(x)
        # Pasa los embeddings por el codificador posicional
        p_encoded_x = self.pos_encoder(embedded_x)
        # Genera queries, keys y values para la atención
        k = self.key(p_encoded_x)
        q = self.query(p_encoded_x)
        v = self.value(p_encoded_x)
        # Calcula los scores de atención como producto punto entre queries y keys
        w = q @ k.transpose(-2, -1) * k.shape[-1] ** -0.5  # Query * Key / normalización
        # Aplica softmax a los scores de atención para obtener probabilidades
        w = F.softmax(w, dim=-1)
        # Multiplica los pesos de atención por los valores para obtener la salida
        out = w @ v
        return out

En el código anterior, la clase `Head` es una implementación de un mecanismo de *self-attention*. Primero convierte cada token de entrada en un vector denso mediante una capa de embedding, y luego le añade información posicional mediante una capa de codificación posicional. Esto permite que el modelo tenga conocimiento sobre la posición de cada token en la secuencia.

Luego, la clase `Head` crea tres proyecciones lineales distintas `(key, query y value)` usando las capas `nn.Linear` definidas en `__init__`. Estas se obtienen multiplicando los datos de entrada con tres matrices de pesos distintas.

Los *scores de atención* se calculan tomando el producto punto entre las proyecciones query y key, y luego se escalan dividiendo por la raíz cuadrada de la dimensión de los keys. Esto estabiliza los gradientes durante el entrenamiento. Luego se aplica una función softmax para convertir los scores en probabilidades que sumen 1.

Finalmente, la salida se obtiene haciendo una suma ponderada de los valores, donde los pesos son los scores de atención. Esta salida representa una combinación de los datos de entrada, influenciada por la relación entre todos los pares de tokens.


In [ ]:
# Instancia la clase Head con la dimensión del embedding y el tamaño del vocabulario como parámetros
transformer = Head(n_embd, vocab_size)

# Pasa los datos de entrada por el modelo transformer para obtener la salida
# Este proceso incluye aplicar embedding, codificación posicional y atención automática
out = transformer(input_data)

# Imprime la forma del tensor de salida
# La forma nos da una idea de cómo se transformaron los datos a través del modelo
print("Forma de la salida:", out.shape)

# Muestra el tensor de salida
# Esta salida representa los datos transformados después de aplicar embedding, codificación posicional y self-attention
print("Salida:", out)

### **Transformers en PyTorch**

En esta parte, aprenderemos cómo crear modelos tipo Transformer utilizando la biblioteca `nn.torch`.

Este bloque de código crea una instancia del modelo Transformer del módulo `nn` (redes neuronales) en PyTorch. El parámetro `nhead` especifica el número de cabeceras en el mecanismo de atención multicabecera, que es un componente crucial de la arquitectura Transformer. En este caso, se establece en 16.

El parámetro `num_encoder_layers` determina el número de capas del codificador en el modelo Transformer. Aquí, se establece en 12.


In [ ]:
transformer_model = nn.Transformer(nhead=16, num_encoder_layers=12)

Estas dos líneas crean tensores aleatorios para representar las secuencias de entrada (`source`) y salida (`target`) para el modelo Transformer.

`src` representa 10 secuencias de entrada, cada una con una longitud de 32 y una dimensión de características de 512.
`tgt` representa 20 secuencias de salida, cada una con una longitud de 32 y una dimensión de características de 512.
En el contexto de tareas de secuencia a secuencia, las secuencias fuente son los datos de entrada (por ejemplo, frases en un idioma), y las secuencias destino son la salida deseada (por ejemplo, frases correspondientes en otro idioma).


In [ ]:
src = torch.rand((10, 32, 512))
tgt = torch.rand((20, 32, 512))

Luego, pasa los tensores de entrada y salida a través del modelo Transformer. La variable `out` contendrá la salida del modelo, que debería tener la misma forma que el tensor `tgt` ((20, 32, 512)). Esta salida puede procesarse más adelante o utilizarse para tareas posteriores, como calcular una función de pérdida durante el entrenamiento o generar texto durante la inferencia.

In [ ]:
out = transformer_model(src, tgt)

#### **Atención multiCabecera (MultiHead Attention)**

`nn.MultiheadAttention` es un módulo en PyTorch que implementa el mecanismo de auto-atención multicabecera (*multi-head self-attention*), un componente clave de la arquitectura Transformer. Este mecanismo de atención permite al modelo enfocarse en diferentes partes de la secuencia de entrada simultáneamente, capturando diversas dependencias contextuales y mejorando su capacidad para procesar patrones complejos del lenguaje natural.

El módulo `nn.MultiheadAttention` tiene tres entradas principales: `query`, `key` y `value` como se ilustra a continuación:

<p style="text-align:center">
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMSkillsNetwork-AI0201EN-Coursera/MultiHeadAttention.png" width="300" alt="MultiHead"/>
</p>

El mecanismo de atención multicabecera funciona dividiendo primero las entradas `query`, `key` y `value` en múltiples "cabeceras", cada una con su propio conjunto de pesos entrenables. Este proceso permite que el modelo aprenda diferentes patrones de atención en paralelo.

Las salidas de todas las cabeceras se concatenan y se pasan a través de una capa lineal, conocida como proyección de salida, para combinar la información aprendida por cada cabecera. Esta salida final representa la secuencia enriquecida contextualmente que puede ser utilizada por las siguientes capas del modelo Transformer.

In [ ]:
# Dimensión del embedding
embed_dim = 4
# Número de cabeceras de atención
num_heads = 2
print("Debería ser cero:", embed_dim % num_heads)
# Inicializa MultiheadAttention
multihead_attn = nn.MultiheadAttention(embed_dim=embed_dim, num_heads=num_heads, batch_first=False)

In [ ]:
seq_length = 10  # Longitud de la secuencia
batch_size = 5   # Tamaño del lote
query = torch.rand((seq_length, batch_size, embed_dim))
key = torch.rand((seq_length, batch_size, embed_dim))
value = torch.rand((seq_length, batch_size, embed_dim))
# Realiza atención multicabecera
attn_output, _ = multihead_attn(query, key, value)
print("Forma de la salida de atención:", attn_output.shape)

#### **TransformerEncoderLayer y TransformerEncoder**

`TransformerEncoderLayer` y `TransformerEncoder` son componentes esenciales de la arquitectura Transformer en PyTorch. Estos componentes trabajan en conjunto para crear una red neuronal con múltiples capas basadas en atención.

#### `TransformerEncoderLayer`:

Esta es una única capa de codificación en la arquitectura Transformer, que consta de dos subcapas principales como se muestra a continuación: la capa de auto-atención multicabecera (*Multi-head Self-Attention*) y la red neuronal alimentada hacia adelante (*Feed-Forward Network* o FFN). Cada una de estas subcapas va seguida de una conexión residual y una normalización por capas (*Layer Normalization*).

<p style="text-align:center">
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMSkillsNetwork-AI0201EN-Coursera/TrLayer.png" width="200" alt="TrLayer"/>
</p>



### `TransformerEncoder`:

El `TransformerEncoder` es una pila de múltiples instancias de `TransformerEncoderLayer`. El codificador consta de `N` capas idénticas. `N` puede ajustarse según la complejidad deseada del modelo.

El codificador toma una secuencia de entrada, le aplica codificación posicional y la pasa secuencialmente por cada capa del codificador. Esto permite al modelo aprender representaciones jerárquicas de la secuencia, capturando dependencias tanto locales como a largo plazo.

`TransformerEncoder` acepta los siguientes parámetros:

* `src` (obligatorio): La secuencia que se le entrega al codificador.
* `mask` (opcional): El parámetro `mask` se usa para restringir el mecanismo de atención a ciertas posiciones de la secuencia de entrada. Es un tensor binario con la misma forma que la entrada. Un valor de 1 indica que la atención está permitida, y 0 indica que debe ser ignorada. Es útil para enmascaramiento triangular donde cada posición solo puede atender a las anteriores.
* `src_key_padding_mask` (opcional): Este parámetro indica qué posiciones en la entrada corresponden a tokens de padding. Es un tensor binario con forma `(batch_size, sequence_length)`. Un valor de 1 indica un token válido; un 0 indica padding. Ayuda a ignorar tokens de relleno y enfocarse en partes significativas.

In [ ]:
# Dimensión del embedding
embed_dim = 4
# Número de cabeceras de atención
num_heads = 2
# Verifica si la dimensión del embedding es divisible por el número de cabeceras
# Imprime: "debería ser cero", embed_dim % num_heads
# Número de capas del codificador
num_layers = 6
# Inicializa la capa del codificador con la dimensión del embedding y número de cabeceras especificado
encoder_layer = nn.TransformerEncoderLayer(d_model=embed_dim, nhead=num_heads)
# Construye el codificador Transformer apilando la capa del codificador 6 veces
transformer_encoder = nn.TransformerEncoder(encoder_layer, num_layers=num_layers)

Ahora probémoslo con una entrada aleatoria:


In [ ]:
# Define longitud de secuencia como 10 y tamaño del lote como 5 para los datos de entrada
seq_length = 10  # Longitud de la secuencia
batch_size = 5   # Tamaño del lote
# Genera un tensor aleatorio para simular embeddings de entrada para el codificador Transformer
x = torch.rand((seq_length, batch_size, embed_dim))
# Aplica el codificador Transformer a la entrada
encoded = transformer_encoder(x)
# Muestra la forma del tensor codificado para verificar la transformación
print("Forma del tensor codificado:", encoded.shape)

#### Ejercicios

1. **Desglose del escalado en la atención**: Revisa la expresión

   $$
     \text{Atención}(q,K,V) \;=\;\mathrm{softmax}\!\bigl(\tfrac{qK^T}{\sqrt{d}}\bigr)\,V
   $$

   y argumenta por qué el factor $1/\sqrt{d}$ mantiene la estabilidad numérica. Compara qué sucede en la distribución de pesos de atención si omites esa normalización o si usas un factor distinto.

2. **Complejidad secuencial vs. paralela**: Analiza el coste computacional de calcular $\mathrm{softmax}(QK^T)V$ procesando un token a la vez frente a hacerlo con toda la matriz $Q$. Estima el número de multiplicaciones y sumas necesarias en cada caso y reflexiona sobre el impacto en tiempo de entrenamiento cuando el tamaño del lote o la longitud de la secuencia crecen.

3. **Atención multicabecera en perspectiva**: Partiendo de la idea de dividir el embedding en varias "cabecera" independientes, razona cómo cambia la capacidad de captura de relaciones cruzadas cuando aumentas o disminuyes el número de cabeceras (manteniendo fijo el tamaño total de $d$). ¿En qué casos podría convenir usar pocas cabeceras de gran tamaño frente a muchas cabeceras de menor dimensión?

4. **Análisis manual de un ejemplo reducido**: Con un vocabulario de 4-6 palabras y vectores one-hot sencillos, elabora a mano las matrices $K$, $V$ y un vector consulta $q$. Calcula paso a paso $\mathrm{softmax}(qK^T)$ y el producto resultante con $V$. Reflexiona sobre cómo se distribuyen los pesos de atención y qué token "gana" en cada caso.

5. **Efecto de la temperatura en softmax**: Imagina añadir un parámetro de temperatura $T$ en la fórmula:

   $$
     \mathrm{softmax}\!\Bigl(\tfrac{qK^T}{T}\Bigr)\,V.
   $$

   Discute cómo los valores de $T<1$ o $T>1$ afectan la diversidad del resultado y cuándo podría ser deseable afinar $T$ para tareas de traducción frente a tareas de generación creativa.

6. **Comparativa one-hot vs. embeddings aprendidos**: Contrasta, a nivel conceptual, la rigidez de trabajar con representaciones one-hot frente a vectores de embedding entrenables. Señala ventajas y desventajas de cada enfoque en términos de capacidad de generalización, uso de memoria y facilidad de interpretación.

7. **Interpretación geométrica en espacio 3D**: Imagina proyectar tus vectores de embedding en un espacio de tres dimensiones y representarlos gráficamente. Describe cómo podrías identificar clusters de tokens similares, qué relaciones semánticas podrían surgir y qué limitaciones tendría esta visualización.

8. **Atención como mecanismo de peso dinámico**: Plantea una comparación teórica entre la atención (que asigna pesos distintos a cada token) y métodos basados en ventanas fijas o contextos de tamaño constante. ¿Cómo influye la atención en la capacidad de modelar dependencias de largo alcance?

9. **Impacto del tamaño de bloque ($block\_size$)**: Reflexiona sobre cómo la longitud máxima de contexto (p. ej. 16 tokens) condiciona la memoria del modelo. Discute qué problemas podría haber al aumentar o reducir ese límite y cómo afectaría tanto a la complejidad computacional como a la calidad de las traducciones.

10. **Diseño de un experimento de evaluación**: Imagina que dispones de un pequeño corpus bilingüe. Propón una manera de medir el efecto de tu implementación de atención (matricial) frente a una simple traducción palabra por palabra. Esboza cómo recogerías métricas cuantitativas (distancia de Levenshtein, exact match...) y qué análisis cualitativo acompañarías.

11. **Dimensiones y formas**: Analiza detenidamente las dimensiones de las tensiones que atraviesan el método `forward`. A partir de un tensor de entrada de forma `(seq_len,)`, razona el tamaño de cada paso: embedding, codificación posicional, proyecciones de `key`, `query` y `value`, cálculo de scores y salida final.

12. **Influencia de la normalización**: Reflexiona sobre el término de escalado `k.shape[-1]**-0.5` en el producto `q @ k.transpose`. ¿Qué ocurriría con la distribución de los pesos de atención si lo omites o si cambias su exponente? Justifica el impacto en estabilidad de gradientes y en la probabilidad resultante.

13. **Máscaras de atención**: Imagina que quisieras impedir que ciertos tokens vean información futura (como en decoders autoregresivos). Describe cómo incorporarías una "máscara triangular" a la matriz de scores antes de aplicar `softmax`, y discute su efecto en la generación de dependencias.

14. **Extensión a Multi-head**: Propón un esquema para dividir la proyección `n_embd -> n_embd` en varias cabeceras de atención más pequeñas. Indica cómo alterarías las capas lineales y cómo combinarías las salidas de cada cabecera para obtener el vector final.

15. **Codificación posicional aprendible vs. fija** :Contrasta las ventajas e inconvenientes de usar un buffer fijo de codificaciones (`register_buffer`) con la opción de aprender un embedding posicional mediante un `nn.Embedding`. ¿En qué situaciones podría ser más beneficioso cada enfoque?

16. **Regularización dentro de la atención**: Discute el papel que podría tener un `Dropout` en la matriz de pesos `w` o sobre las salidas `out`. ¿Cómo afectaría esta técnica al sobreajuste y a la diversidad de patrones de atención durante entrenamiento y en inferencia?

17. **Análisis de casos límite**: Describe qué sucede en dos escenarios opuestos:

   * **Secuencias muy cortas**: ¿cómo se comporta la atención cuando `seq_len` es 1 o 2?
   * **Secuencias muy largas**: reflexiona sobre la complejidad computacional y el uso de memoria al crecer `seq_len`, y cómo podrías mitigar este coste.

18. **Atención causal vs. completa**: Debate las diferencias conceptuales entre usar una atención "completa" donde cada token atiende a todos los demás, frente a una "causal" que limita la visibilidad hacia tokens posteriores. ¿Qué tipos de tareas requieren uno u otro?

19. **Inspección cualitativa de `w`**  Plantea cómo, para una frase corta, podrías extraer e interpretar los valores de la matriz `w` (por ejemplo, como un mapa de calor), y qué conclusiones semánticas o de posición podrías derivar de sus patrones de activación.

20. **Variantes de codificación posicional** Más allá de las funciones trigonométricas, sugiere dos alternativas para generar codificaciones posicionales (por ejemplo, polinomios de bajo grado, codificaciones randómicas), y discute teóricamente cómo cambiarían la inyectividad y la capacidad del modelo para capturar orden.



In [ ]:
# Tus respuestas